# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = "../output_data/citiesweatherdata.csv"
cities_df = pd.read_csv(csv_path)
cities_df


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,torbay,47.67,-52.73,59.00,100,90,25.28,CA,1597815709
1,punta arenas,-53.15,-70.92,33.80,74,73,9.17,CL,1597815709
2,kapaa,22.08,-159.32,78.80,78,90,19.46,US,1597815709
3,faanui,-16.48,-151.75,78.17,76,37,23.04,PF,1597815544
4,port alfred,-33.59,26.89,54.23,69,54,23.69,ZA,1597815710
...,...,...,...,...,...,...,...,...,...
566,mollendo,-17.02,-72.01,55.44,79,72,6.60,PE,1597815642
567,susaki,33.37,133.28,89.60,75,20,11.41,JP,1597815642
568,nahrin,36.06,69.13,79.77,23,0,10.27,AF,1597815643
569,fort oglethorpe,34.95,-85.26,75.20,73,1,1.41,US,1597815643


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key = g_key)

# Store 'Lat' and 'Lng' into  locations
locations = cities_df[["Latitude", "Longitude"]].astype(float)

# Store 'humidity' as weight
humid_data = cities_df['Humidity'].astype(float)

In [4]:
# Create heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humid_data, dissipating=False, max_intensity=100, point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_df = cities_df.loc[(cities_df["Wind Speed"] <= 10) & (cities_df["Cloudiness"] == 0) & \
                                   (cities_df["Max Temp"] >= 70) & (cities_df["Max Temp"] <= 80)].dropna()
narrowed_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,san quintin,30.48,-115.95,77.47,71,0,4.99,MX,1597815711
113,alice springs,-23.70,133.88,75.00,18,0,5.82,AU,1597815508
137,mweka,-4.85,21.57,79.29,65,0,3.27,CD,1597815564
180,alanya,36.54,32.00,75.20,57,0,2.24,TR,1597815574
246,lanzhou,36.06,103.79,73.40,38,0,6.71,CN,1597815296
262,kindu,-2.95,25.95,79.12,65,0,3.62,CD,1597815592
404,los llanos de aridane,28.66,-17.92,71.01,88,0,8.05,ES,1597815617
406,oktyabrskoye,43.06,44.74,71.01,72,0,2.24,RU,1597815617
421,bolu,40.67,31.58,76.05,34,0,0.38,TR,1597815619
436,jacareacanga,-6.22,-57.75,73.42,45,0,0.87,BR,1597815622


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = narrowed_df.loc[:,["City", "Country", "Latitude", "Longitude"]].rename(
    columns = {'Latitude':'Lat', 'Longitude':'Lng'})

# Add a "Hotel Name" column to the DataFrame.
hotel_df['Hotel Name'] = ''

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
9,san quintin,MX,30.48,-115.95,
113,alice springs,AU,-23.70,133.88,
137,mweka,CD,-4.85,21.57,
180,alanya,TR,36.54,32.00,
246,lanzhou,CN,36.06,103.79,
262,kindu,CD,-2.95,25.95,
404,los llanos de aridane,ES,28.66,-17.92,
406,oktyabrskoye,RU,43.06,44.74,
421,bolu,TR,40.67,31.58,
436,jacareacanga,BR,-6.22,-57.75,


In [39]:
lats = hotel_df['Lat'].tolist()
longs = hotel_df['Lng'].tolist()
best_cities = zip(lats,longs)
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_name = []

#Iterate through cities list and call the api to get hotel info
for location in best_cities:
    print()
    params = {
        'location': str(location[0]) + ',' + str(location[1]),
        'radius': 5000,
        'type': 'lodging',
        'keyword': 'hotel',
        'key': g_key
    }
    response = requests.get(base_url, params = params).json()
    try:
        hotel_name.append(response['results'][0]['name'])
    except:
        hotel_name.append('')
        
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
        

        
        
hotel_df["Hotel"] = hotel_name
hotel_df

,City,Country,Lat,Lng,Hotel Name,Hotel
9,san quintin,MX,30.48,-115.95,,Old Mill Hotel
113,alice springs,AU,-23.70,133.88,,DoubleTree by Hilton Hotel Alice Springs
137,mweka,CD,-4.85,21.57,,
180,alanya,TR,36.54,32.00,,Diamond Hill Resort Hotel
246,lanzhou,CN,36.06,103.79,,Sapphire Grand Hotel
262,kindu,CD,-2.95,25.95,,Valencia City Hotel
404,los llanos de aridane,ES,28.66,-17.92,,Hotel Hacienda de Abajo
406,oktyabrskoye,RU,43.06,44.74,,Aleksandrovskiy
421,bolu,TR,40.67,31.58,,Gazelle Resort&SPA
436,jacareacanga,BR,-6.22,-57.75,,HOTEL VIANA
